# Example 1

Example using a dataset collected by a single trifluoride boron detector manufactured by Hydroinnova, LLC.

Goal: Demonstrate the use of `CRNPy` to convert raw sensor data into surface soil water storage following common steps in the scientific literature.

Author: Andres Patrignani

In [8]:
import crnpy
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
output_notebook()


Loading BokehJS ...

In [3]:
# Parameter values
par = {'number_detectors':1,
       'count_time': 3600,
       'Pref':980, 
       'L':130, 
       'Aref':0,
       'N0':3486, 
       'Wlat':0.03,
       'Wsoc':0.01, 
       'bulk_density':1.33,
       'utc_offset':-6,
       'nmdb_station':'IRKT'}



In [4]:
# Define file name, site-specific variables, and load data
filename = 'hydroinnova_stationary_bf/hydroinnova.csv'
skip_rows = 21

# Variables in file
variables = {'timestamp':[2],
             'temperature':[7],
             'humidity':[8],
             'pressure':[3],
             'counts_raw':[10],
             'counts_time':[11]}

df = pycrns.read_data(filename,skip_rows,variables,round_time='H')

In [10]:
# Load file from Radiation detection technologies
filename = 'rdt_stationary_lifoil/rdt.csv'
variables = ['timestamp','pressure','RH', 'T','DP','BattVolt','counts_1','counts_2','counts_3','counts_total'];
df = pd.read_csv(filename,names=variables)
df

,timestamp,pressure,RH,T,DP,BattVolt,counts_1,counts_2,counts_3,counts_total
0,4/10/2020 9:17,983.8,29.0,9.6,-7.4,14.4,848,716,742,2306
1,4/10/2020 10:17,982.3,25.0,10.9,-7.9,14.3,436,720,796,1952
2,4/10/2020 11:17,980.8,25.0,11.5,-7.4,13.7,777,792,708,2277
3,4/10/2020 12:17,979.5,22.0,13.6,-7.3,13.9,820,786,714,2320
4,4/10/2020 13:17,977.2,23.0,14.5,-6.0,13.7,844,707,734,2285
...,...,...,...,...,...,...,...,...,...,...
1649,6/18/2020 3:03,973.0,81.0,22.4,19.0,12.4,846,811,11706,13363
1650,6/18/2020 4:03,973.0,80.0,22.8,19.2,12.6,820,818,13974,15612
1651,6/18/2020 5:03,973.4,70.0,25.0,19.3,13.3,853,805,13004,14662
1652,6/18/2020 6:03,973.5,64.0,26.7,19.5,13.6,857,720,4352,5929


In [6]:
df.pipe(pycrns.fill_counts_v2,[['counts_1','counts_2','counts_3']],['counts_time'])

,timestamp,pressure,temperature,humidity,counts_raw,counts_time
0,2020-04-10 17:00:00,984.21,7.7,32.7,2115.0,3600.0
1,2020-04-10 18:00:00,982.98,8.9,29.0,2198.0,3600.0
2,2020-04-10 19:00:00,981.50,10.8,28.9,2162.0,3600.0
3,2020-04-10 20:00:00,979.95,11.3,28.1,2177.0,3600.0
4,2020-04-10 21:00:00,978.16,13.5,24.9,2212.0,3600.0
...,...,...,...,...,...,...
1659,2020-06-18 19:00:00,973.78,34.0,41.9,2157.0,3600.0
1660,2020-06-18 20:00:00,973.52,34.2,41.6,2162.0,3600.0
1661,2020-06-18 21:00:00,973.20,33.5,42.5,2211.0,3600.0
1662,2020-06-18 22:00:00,972.70,33.4,42.6,2247.0,3600.0


In [ ]:
# Correct neutron counts
#data = pycrns.load_data(filename,skip_rows,variables) 
#data = pycrns.fill_counts(data,par)

date = pycrns(filename,skip_rows,variables,par)
#data = data.fill_counts(par)
data = pycrns.fill_missing_atm(data)
data = pycrns.compute_total_counts(data)
data = pycrns.get_incoming_neutron_flux(data,par)
data = pycrns.atm_correction(data,par)
data = pycrns.smooth_counts(data)
data = pycrns.counts_to_vwc(data,par)
data = pycrns.sensing_depth(data,par,dist=[5,50,150])
data = pycrns.storage(data,T=5)

In [ ]:
f = figure(x_axis_type='datetime', width=500, height=300)
f.line(data['timestamp'], data['counts_total_raw'], color='green', legend_label='N raw')
f.line(data['timestamp'], data['counts_total_corrected'], color='black', legend_label='N corrected')
f.line(data['timestamp'], data['counts_total_filtered'], color='tomato', legend_label='N filtered')
f.yaxis.axis_label = 'Neutron counts (cph)'
show(f)

In [ ]:
f = figure(x_axis_type='datetime', width=500, height=300)
f.line(data['timestamp'], data['temperature'])
f.line(data['timestamp'], data['humidity'])
show(f)

In [ ]:
f = figure(x_axis_type='datetime', width=500, height=300)
f.line(data['timestamp'], data['pressure'])
f.yaxis.axis_label = 'Atmospheric pressure (mbar)'
show(f)

In [ ]:
f = figure(x_axis_type='datetime', width=500, height=300)
f.line(data['timestamp'], data['vwc'])
f.yaxis.axis_label = 'Volumetric water content (cm^3/cm^3)'
show(f)

In [ ]:
f = figure(x_axis_type='datetime', width=500, height=300)
f.line(data['timestamp'], data['sensing_depth_D86_5m'], color='tomato', legend_label='5 m')
f.line(data['timestamp'], data['sensing_depth_D86_150m'], color='blue', legend_label='150 m')
f.yaxis.axis_label = 'Sensing depth (cm)'
f.legend.location = 'bottom_left'
show(f)

In [ ]:
f = figure(x_axis_type='datetime', width=500, height=300)
f.line(data['timestamp'], data['storage_surface'])
f.yaxis.axis_label = 'Storage (cmm)'
show(f)

In [ ]:
f = figure(x_axis_type='datetime', width=500, height=300)
f.line(data['timestamp'], data['storage_rootzone'])
f.yaxis.axis_label = 'Storage (cmm)'
show(f)